# Assignment 2
## Twitch Recommendation System
Authors: Alex Cojocaru, Kyle Jorrin, Diego Otero-Caldwell, Alec Drumm

Research articles to get started:\
[YouTube Recommendation Algorithm](https://dl.acm.org/doi/abs/10.1145/1864708.1864770)\
[Overview Of Recommender Systems](https://search-library.ucsd.edu/permalink/01UCS_SDI/1vtf07t/cdi_doaj_primary_oai_doaj_org_article_e1fff15ae9b64b96915b66bc5dc81ac5)\
[FPMC](https://dl.acm.org/doi/abs/10.1145/1772690.1772773)\
...




In [ ]:
# If first time, run this script to set up virtual environment
# Requires python3.11
!chmod +x scripts/setup_env.sh
!./scripts/setup_env.sh

Data link: https://cseweb.ucsd.edu/~jmcauley/datasets.html#twitch

Start and stop times are provided as integers and represent periods of 10 minutes. Stream ID could be used to retrieve a single broadcast segment from a streamer (not used in our work).

    User ID (anonymized)
    Stream ID
    Streamer username
    Time start
    Time stop

[Original research paper of the data](https://search-library.ucsd.edu/permalink/01UCS_SDI/1vtf07t/cdi_unpaywall_primary_10_1145_3460231_3474267)


Load in the data

In [8]:
# Imports
import pandas as pd
import numpy as np

In [11]:
# Data with header names
data = pd.read_csv('100k_a.csv', names=['user_id', 'stream_id', 'streamer_username', 'time_start', 'time_stop'])
data.head()

,user_id,stream_id,streamer_username,time_start,time_stop
0,1,33842865744,mithrain,154,156
1,1,33846768288,alptv,166,169
2,1,33886469056,mithrain,587,588
3,1,33887624992,wtcn,589,591
4,1,33890145056,jrokezftw,591,594


### Training the model (Part 3)
#### From section 4.2 of the paper

In [24]:
import pandas as pd
from scipy.sparse import csr_matrix

# Group by user_id and streamer_username, count interactions
interaction_counts = data.groupby(['user_id', 'streamer_username']).size().reset_index(name='count')

# Map user_id and streamer_username to indices for the matrix
user_ids = interaction_counts['user_id'].unique()
streamer_usernames = interaction_counts['streamer_username'].unique()

user_to_idx = {user: idx for idx, user in enumerate(user_ids)}
streamer_to_idx = {streamer: idx for idx, streamer in enumerate(streamer_usernames)}

# Prepare data for sparse matrix
rows = interaction_counts['user_id'].map(user_to_idx)
cols = interaction_counts['streamer_username'].map(streamer_to_idx)
values = interaction_counts['count']

# Create sparse matrix (users x streamers)
user_streamer_matrix = csr_matrix((values, (rows, cols)), shape=(len(user_ids), len(streamer_usernames)))
user_streamer_matrix.shape

(100000, 162625)

In [25]:
# Replace surprise with implicit
import implicit

# Train ALS model
model = implicit.als.AlternatingLeastSquares(factors=50)
model.fit(user_streamer_matrix)

# Recommend for user 0
recommendations, scores = model.recommend(0, user_streamer_matrix[0])
print("Recommended items:", recommendations)
print("Scores:", scores)

100%|██████████| 15/15 [00:08<00:00,  1.76it/s]

Recommended items: [  50 1931   57  961  196  181 2605   56  180 2983]
Scores: [0.7568445  0.61852187 0.6125231  0.5044     0.48565707 0.4336766
 0.42903388 0.42077616 0.41476846 0.36341152]
